In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
eva=pd.read_csv('Number_Detection/number_csv/data.csv')


In [4]:
display(eva.head())

,filename,height,left,top,width,label
0,1.png,219,246,77,81,1
1,1.png,219,323,81,96,9
2,2.png,32,77,29,23,2
3,2.png,32,98,25,26,3
4,3.png,15,17,5,8,2


In [2]:
print(eva.isnull().sum())
# print(eva.type())

filename    0
height      0
left        0
top         0
width       0
label       0
dtype: int64


In [3]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('/home/eva/jupyter_home/Number_Detection/number_csv/data.csv')

# 'xmin', 'ymin', 'xmax', 'ymax' 컬럼 계산
df['xmin'] = df['left']
df['ymin'] = df['top']
df['xmax'] = df['left'] + df['width']
df['ymax'] = df['top'] + df['height']

# 새로운 CSV 파일로 저장
df_xyxy = df[['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'label']]
df_xyxy.to_csv('bounding_boxes_xyxy.csv', index=False)

# 결과 출력
print(df_xyxy)


        filename  xmin  ymin  xmax  ymax  label
0          1.png   246    77   327   296      1
1          1.png   323    81   419   300      9
2          2.png    77    29   100    61      2
3          2.png    98    25   124    57      3
4          3.png    17     5    25    20      2
...          ...   ...   ...   ...   ...    ...
73252  33401.png    34     6    59    46      2
73253  33401.png    61     4    86    44      2
73254  33402.png    35    10    42    35      1
73255  33402.png    44     8    59    33      6
73256  33402.png    62     9    79    34      9

[73257 rows x 6 columns]


In [4]:
display(df_xyxy.head())

,filename,xmin,ymin,xmax,ymax,label
0,1.png,246,77,327,296,1
1,1.png,323,81,419,300,9
2,2.png,77,29,100,61,2
3,2.png,98,25,124,57,3
4,3.png,17,5,25,20,2


In [19]:
print(df_xyxy['label'].unique())

[1 9 2 3 5 8 7 4 6 0]


In [5]:
from sklearn.model_selection import train_test_split
train_dir, validation_dir =train_test_split(df_xyxy,
                                           test_size=0.2,
                                           stratify=df_xyxy['label'])

In [21]:
cd Number_Detection

[Errno 2] No such file or directory: 'Number_Detection'
/home/eva/jupyter_home/Number_Detection


In [12]:
# PASCAL VOC 형태로 만들어야 함 
# images -> 현재 train/ 안에 저장되어 있음 => pascal_voc_data/JPEGImages/ 로 복사
# pascal_voc_data/Annotations/ 에 annotation (.xml) 파일을 생성

import cv2
import os

main_dir='/home/eva/jupyter_home/Number_Detection'
def make_xml(df):
    grouped=df.groupby('filename')
    for filename, group in grouped:
        xml_name = os.path.splitext(filename)[0] + '.xml'
        path = os.path.join(main_dir, 'train', filename)
        img = cv2.imread(path)
        
        if img is not None:
            h, w, c = img.shape
        else:
            print(f"이미지 로딩 실패: {filename}")
            continue
        with open(os.path.join(main_dir,'VOCdevkit/voc2007/Annotations',xml_name), 'w' ) as f:
            f.write(f'''
            <annotation>
                <folder>JPEGImages</folder>
                <filename>{filename}</filename>
                <path>{path}</path>
                <size>
                    <width>{w}</width>
                    <height>{h}</height>
                    <depth>{c}</depth>
                </size>''')
            for _, row in group.iterrows():
                filename=row['filename']
                xml_name=row['filename'].split('.')[0]+'.xml'
                xmin=row['xmin']
                xmax=row['xmax']
                ymin=row['ymin']
                ymax=row['ymax']
                label=row['label']
                f.write(f'''
                    <object>
                        <name>{label}</name>
                        <bndbox>
                            <xmin>{xmin}</xmin>
                            <ymin>{ymin}</ymin>
                            <xmax>{xmax}</xmax>
                            <ymax>{ymax}</ymax>
                        </bndbox>
                    </object>''')
            f.write('</annotation>')

make_xml(df)

In [ ]:
#이미지 JPEGImages 로cp하기 ->Number_Detection/VOCdevkit/voc2007/JPEGImages 여기로 
 cp -r /home/eva/jupyter_home/Number_Detection/train . 
#그리고 imagesets을 txt로 마들어서 가져온다 train/val 80:20으로 해서 

In [12]:
#tf_recor을 생성하기 위해 research/models안에 있는 object_detection으로 가야하기 때문에 pwd 했을때 아래와 같은 경로에 있어야 한다  
#/home/eva/jupyter_home/TF2_Object_Detection/models/research
cd research

/home/eva/jupyter_home/TF2_Object_Detection/models/research


In [15]:
import os
import tensorflow as tf
import xml.etree.ElementTree as ET

# VOC 클래스 목록 (20 classes)
VOC_CLASSES = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

class_name_to_id = {name: i for i, name in enumerate(VOC_CLASSES)}

def parse_voc_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.find("filename").text
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    bboxes = []
    labels = []

    for obj in root.findall("object"):
        name = obj.find("name").text
        label = class_name_to_id[name]

        bbox = obj.find("bndbox")
        xmin = int(float(bbox.find("xmin").text))
        ymin = int(float(bbox.find("ymin").text))
        xmax = int(float(bbox.find("xmax").text))
        ymax = int(float(bbox.find("ymax").text))

        bboxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)

    return filename, width, height, bboxes, labels

def create_tf_example(image_path, xml_path):
    filename, width, height, bboxes, labels = parse_voc_xml(xml_path)

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()

    xmins = [box[0] / width for box in bboxes]
    ymins = [box[1] / height for box in bboxes]
    xmaxs = [box[2] / width for box in bboxes]
    ymaxs = [box[3] / height for box in bboxes]

    classes_text = [VOC_CLASSES[label].encode('utf8') for label in labels]
    classes = labels

    feature = {
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode('utf8')])),
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }

    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example

def generate_tfrecord(voc_dir, split_txt, output_path):
    annotation_dir = os.path.join(voc_dir, 'Annotations')
    image_dir = os.path.join(voc_dir, 'JPEGImages')

    with tf.io.TFRecordWriter(output_path) as writer:
        with open(split_txt, 'r') as f:
            lines = f.read().strip().splitlines()

        for img_id in lines:
            xml_path = os.path.join(annotation_dir, f"{img_id}.xml")
            img_path = os.path.join(image_dir, f"{img_id}.png")

            if os.path.exists(xml_path) and os.path.exists(img_path):
                example = create_tf_example(img_path, xml_path)
                writer.write(example.SerializeToString())

VOC_ROOT_DIR = '/home/eva/jupyter_home/Number_Detection'

# 실행
generate_tfrecord(
    voc_dir=VOC_ROOT_DIR + "/VOCdevkit/voc2007",
    split_txt=VOC_ROOT_DIR + "/VOCdevkit/voc2007/ImageSets/Main/train.txt",
    output_path=VOC_ROOT_DIR + "/num_voc2007_train.tfrecord" #만들어 지는 파일  
)

generate_tfrecord(
    voc_dir=VOC_ROOT_DIR + "/VOCdevkit/voc2007",
    split_txt=VOC_ROOT_DIR + "/VOCdevkit/voc2007/ImageSets/Main/val.txt",
    output_path=VOC_ROOT_DIR + "/num_voc2007_val.tfrecord"   #만들어 지는 파일
)

In [38]:
cd Number_Detection

/home/eva/jupyter_home/Number_Detection


In [16]:
IMAGE_SIZE = 512

def parse_tfrecord(example):
    features = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

    parsed = tf.io.parse_single_example(example, features)
    image = tf.image.decode_jpeg(parsed['image/encoded'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])

    xmins = tf.sparse.to_dense(parsed['image/object/bbox/xmin'])
    ymins = tf.sparse.to_dense(parsed['image/object/bbox/ymin'])
    xmaxs = tf.sparse.to_dense(parsed['image/object/bbox/xmax'])
    ymaxs = tf.sparse.to_dense(parsed['image/object/bbox/ymax'])
    labels = tf.sparse.to_dense(parsed['image/object/class/label'])

    boxes = tf.stack([xmins, ymins, xmaxs, ymaxs], axis=-1)

    return {
        "images": image,
        "bounding_boxes": {
            "boxes": boxes,
            "classes": tf.cast(labels, tf.int32),
        }
    }

def load_dataset(tfrecord_path, batch_size=8):
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(parse_tfrecord, 
                          num_parallel_calls=tf.data.AUTOTUNE)
    
    # padded_batch 필수
    dataset = dataset.shuffle(1000).padded_batch(
        batch_size,
        padded_shapes={
            "images": [IMAGE_SIZE, IMAGE_SIZE, 3],
            "bounding_boxes": {
                "boxes": [None, 4],
                "classes": [None]
            }
        },
        drop_remainder=True
    ).prefetch(tf.data.AUTOTUNE)

    return dataset

#데이터 셋 준비 
train_ds = load_dataset("./num_voc2007_train.tfrecord", 
                        batch_size=8)

val_ds = load_dataset("./num_voc2007_val.tfrecord", 
                        batch_size=8)     

In [46]:
!curl -I https://www.kaggle.com


HTTP/2 404 
content-length: 134
content-type: text/html; charset=UTF-8
date: Tue, 13 May 2025 02:46:18 GMT
alt-svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000



In [12]:
from keras_cv.models import RetinaNet
#이미 만들어 놓은 모델을 가지고 와서 만들어야 함! : from_preset() 
#어떤 preset()이 있는지 확인 필요함!
model = RetinaNet.from_preset(
    'resnet50',
    bounding_box_format='xyxy',
    num_classes=10
)


In [13]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-4),
             classification_loss='focal',
             box_loss='smoothl1')

In [17]:
cd Number_Detection

/home/eva/jupyter_home/Number_Detection


In [ ]:
model.fit(train_ds,
          batch_size=16,
          epochs=10,
          verbose=1)


Epoch 1/10
    256/Unknown - 108s 337ms/step - loss: 0.0000e+00 - box_loss: 0.0000e+00 - classification_loss: 0.0000e+00 - percent_boxes_matched_with_anchor: 0.4371

In [27]:
cd Number_Detection

/home/eva/jupyter_home/Number_Detection
